<a href="https://colab.research.google.com/github/Eboselethefirst/credit-card-fraud-detection/blob/main/Classification_With_SparkML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [54]:
!pip install findspark

In [55]:
def warn(*args,**kwargs):
  pass
import warnings as wng
wng.warn= warn
wng.filterwarnings("ignore")


In [56]:
import findspark as fs
fs .init()
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


In [57]:
spark = SparkSession.builder.appName("Classification with SparkML").getOrCreate()

In [58]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/drybeans.csv
!ls

--2026-01-21 12:15:18--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/drybeans.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2484759 (2.4M) [text/csv]
Saving to: ‘drybeans.csv.1’

drybeans.csv.1      100%[===================>]   2.37M  1.74MB/s    in 1.4s    

2026-01-21 12:15:21 (1.74 MB/s) - ‘drybeans.csv.1’ saved [2484759/2484759]

drybeans.csv  drybeans.csv.1  sample_data


In [59]:
df = spark.read.csv("drybeans.csv",header = True,inferSchema=True)
df.show(7)

+-----+---------+---------------+---------------+------------+------------+----------+-------------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-----+
| Area|Perimeter|MajorAxisLength|MinorAxisLength|AspectRation|Eccentricity|ConvexArea|EquivDiameter|     Extent|   Solidity|  roundness|Compactness|ShapeFactor1|ShapeFactor2|ShapeFactor3|ShapeFactor4|Class|
+-----+---------+---------------+---------------+------------+------------+----------+-------------+-----------+-----------+-----------+-----------+------------+------------+------------+------------+-----+
|28395|  610.291|    208.1781167|     173.888747| 1.197191424| 0.549812187|     28715|  190.1410973|0.763922518|0.988855999|0.958027126|0.913357755| 0.007331506| 0.003147289| 0.834222388| 0.998723889|SEKER|
|28734|  638.018|    200.5247957|    182.7344194| 1.097356461| 0.411785251|     29172|  191.2727505|0.783968133|0.984985603|0.887033637|0.953860842| 0.006978659| 0.00356362

In [60]:
df.printSchema()

root
 |-- Area: integer (nullable = true)
 |-- Perimeter: double (nullable = true)
 |-- MajorAxisLength: double (nullable = true)
 |-- MinorAxisLength: double (nullable = true)
 |-- AspectRation: double (nullable = true)
 |-- Eccentricity: double (nullable = true)
 |-- ConvexArea: integer (nullable = true)
 |-- EquivDiameter: double (nullable = true)
 |-- Extent: double (nullable = true)
 |-- Solidity: double (nullable = true)
 |-- roundness: double (nullable = true)
 |-- Compactness: double (nullable = true)
 |-- ShapeFactor1: double (nullable = true)
 |-- ShapeFactor2: double (nullable = true)
 |-- ShapeFactor3: double (nullable = true)
 |-- ShapeFactor4: double (nullable = true)
 |-- Class: string (nullable = true)



In [61]:
beans_df =df.select(["Area","Perimeter","Solidity","roundness","Compactness","Class"])

In [62]:
beans_df.show(7)

+-----+---------+-----------+-----------+-----------+-----+
| Area|Perimeter|   Solidity|  roundness|Compactness|Class|
+-----+---------+-----------+-----------+-----------+-----+
|28395|  610.291|0.988855999|0.958027126|0.913357755|SEKER|
|28734|  638.018|0.984985603|0.887033637|0.953860842|SEKER|
|29380|   624.11|0.989558774|0.947849473|0.908774239|SEKER|
|30008|  645.884|0.976695743|0.903936374|0.928328835|SEKER|
|30140|  620.134| 0.99089325|0.984877069|0.970515523|SEKER|
|30279|  634.927|0.989509804|0.943851783|0.923725952|SEKER|
|30477|  670.033|0.984081369|0.853079869|0.933373552|SEKER|
+-----+---------+-----------+-----------+-----------+-----+
only showing top 7 rows


In [63]:
beans_df.groupBy("Class").count().orderBy('count').show()

+--------+-----+
|   Class|count|
+--------+-----+
|  BOMBAY|  522|
|BARBUNYA| 1322|
|    CALI| 1630|
|   HOROZ| 1928|
|   SEKER| 2027|
|    SIRA| 2636|
|DERMASON| 3546|
+--------+-----+



In [64]:
indexer = StringIndexer(inputCol="Class",outputCol="label")
beans_df= indexer.fit(df).transform(df)

In [65]:
beans_df.groupby(["Class","label"]).count().orderBy("count").show()

+--------+-----+-----+
|   Class|label|count|
+--------+-----+-----+
|  BOMBAY|  6.0|  522|
|BARBUNYA|  5.0| 1322|
|    CALI|  4.0| 1630|
|   HOROZ|  3.0| 1928|
|   SEKER|  2.0| 2027|
|    SIRA|  1.0| 2636|
|DERMASON|  0.0| 3546|
+--------+-----+-----+



In [66]:
assembler= VectorAssembler(inputCols=["Area","Perimeter","Solidity","roundness","Compactness"],outputCol="features")
beans_transformed_df = assembler.transform(beans_df)

In [67]:
beans_transformed_df.select("features","label").show()

+--------------------+-----+
|            features|label|
+--------------------+-----+
|[28395.0,610.291,...|  2.0|
|[28734.0,638.018,...|  2.0|
|[29380.0,624.11,0...|  2.0|
|[30008.0,645.884,...|  2.0|
|[30140.0,620.134,...|  2.0|
|[30279.0,634.927,...|  2.0|
|[30477.0,670.033,...|  2.0|
|[30519.0,629.727,...|  2.0|
|[30685.0,635.681,...|  2.0|
|[30834.0,631.934,...|  2.0|
|[30917.0,640.765,...|  2.0|
|[31091.0,638.558,...|  2.0|
|[31107.0,640.594,...|  2.0|
|[31158.0,642.626,...|  2.0|
|[31158.0,641.105,...|  2.0|
|[31178.0,636.888,...|  2.0|
|[31202.0,644.454,...|  2.0|
|[31203.0,639.782,...|  2.0|
|[31272.0,638.666,...|  2.0|
|[31335.0,635.011,...|  2.0|
+--------------------+-----+
only showing top 20 rows


In [68]:
(training_data,testing_data)=beans_transformed_df.randomSplit([0.7,0.3],seed = 42)

In [69]:
classifier = LogisticRegression(featuresCol="features",labelCol="label")
model = classifier.fit(training_data)

In [70]:
predictions = model.transform(testing_data)

In [71]:
evaluator = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("The accuracy of the ML Model is =:",accuracy)

The accuracy of the ML Model is =: 0.9140055318078953


In [72]:
evaluator = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="weightedPrecision")
precision = evaluator.evaluate(predictions)
print("The precision score is ", precision)

The precision score is  0.914542510879638


In [73]:
evaluator = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="weightedRecall")
recall = evaluator.evaluate(predictions)
print("The recall score is ", recall)

The recall score is  0.9140055318078953


In [74]:
evaluator = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="f1")
f1 = evaluator.evaluate(predictions)
print("The F1 score is ", f1)

The F1 score is  0.9141223602932477


In [75]:
spark.stop()

In [76]:
spark = SparkSession.builder.appName("Iris Flower Classification").getOrCreate()

In [77]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/iris.csv


--2026-01-21 12:26:22--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-BD0231EN-SkillsNetwork/datasets/iris.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4612 (4.5K) [text/csv]
Saving to: ‘iris.csv’

iris.csv            100%[===================>]   4.50K  --.-KB/s    in 0s      

2026-01-21 12:26:23 (1.25 GB/s) - ‘iris.csv’ saved [4612/4612]



In [78]:
!ls

drybeans.csv  drybeans.csv.1  iris.csv	sample_data


In [79]:
iris_df = spark.read.csv("iris.csv",header = True,inferSchema=True)
iris_df.show(7)

+-------------+------------+-------------+------------+-----------+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|
+-------------+------------+-------------+------------+-----------+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|
|          4.9|         3.0|          1.4|         0.2|Iris-setosa|
|          4.7|         3.2|          1.3|         0.2|Iris-setosa|
|          4.6|         3.1|          1.5|         0.2|Iris-setosa|
|          5.0|         3.6|          1.4|         0.2|Iris-setosa|
|          5.4|         3.9|          1.7|         0.4|Iris-setosa|
|          4.6|         3.4|          1.4|         0.3|Iris-setosa|
+-------------+------------+-------------+------------+-----------+
only showing top 7 rows


In [80]:
indexer = StringIndexer(inputCol="Species",outputCol="label")
iris_df= indexer.fit(iris_df).transform(iris_df)

In [82]:
iris_df.groupby("Species","label").count().show()
iris_df.show(1)

+---------------+-----+-----+
|        Species|label|count|
+---------------+-----+-----+
|    Iris-setosa|  0.0|   50|
| Iris-virginica|  2.0|   50|
|Iris-versicolor|  1.0|   50|
+---------------+-----+-----+

+-------------+------------+-------------+------------+-----------+-----+
|SepalLengthCm|SepalWidthCm|PetalLengthCm|PetalWidthCm|    Species|label|
+-------------+------------+-------------+------------+-----------+-----+
|          5.1|         3.5|          1.4|         0.2|Iris-setosa|  0.0|
+-------------+------------+-------------+------------+-----------+-----+
only showing top 1 row


In [83]:
assembler = VectorAssembler(inputCols=["SepalLengthCm","SepalWidthCm","PetalLengthCm","PetalWidthCm"],outputCol="features")
iris_transfomed_df = assembler.transform(iris_df)

In [85]:
(training_data,testing_data) = iris_transfomed_df.randomSplit([0.7,0.3],seed =42)


In [86]:
classifier = LogisticRegression(featuresCol="features",labelCol="label")
model = classifier.fit(training_data)

In [90]:
predictions = model.transform(testing_data)

In [91]:
evaluator1 = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="accuracy")
accuracy = evaluator1.evaluate(predictions)
evaluator2 = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="weightedPrecision")
precision = evaluator2.evaluate(predictions)
evaluator3 = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="weightedRecall")
recall = evaluator2.evaluate(predictions)
evaluator4 = MulticlassClassificationEvaluator(labelCol="label",predictionCol="prediction",metricName="f1")
f1 = evaluator2.evaluate(predictions)

print("The Accuracy is:",accuracy)
print("The Precision Score is:", precision)
print("The Recall Score is :", recall)
print("The F1 score is :",f1)

The Accuracy is: 0.9782608695652174
The Precision Score is: 0.9804347826086957
The Recall Score is : 0.9804347826086957
The F1 score is : 0.9804347826086957
